<a href="https://colab.research.google.com/github/rhodes-byu/cs-stat-180/blob/main/notebooks/05-data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>


# From Mess to Model: Mastering Data Preparation

This notebook accompanies the lecture and provides runnable examples covering:
- Data discovery & profiling  
- Cleaning (missing values, inconsistencies, duplicates, outliers)  
- Transformation (scaling, aggregation)  
- Feature engineering  
- A mini case study for customer churn preparation


## Setup: Loading Files into Colab environment
This method works in any Python environment (like a local Jupyter Notebook or a script) and doesn't require installing special libraries, but it does require you to change the sharing settings of your file.

* **Step 1**: Share the File in Google Drive
Go to your Google Drive, right-click on worldcities.csv, and select Share. In the sharing settings, change the access from "Restricted" to "Anyone with the link".

* **Step 2**: Copy the Link and Extract the File ID
Copy the shareable link. It will look like this:
https://drive.google.com/file/d/SOME_LONG_FILE_ID/view?usp=sharing

The important part is the FILE_ID, which is the long string of characters between /d/ and /view.

* **Step 3**: Construct the Direct Download URL and Read with Pandas
You can use the FILE_ID to create a direct download link. The following Python code does this for you:

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

# Paste the FILE_ID you extracted from the shareable link
file_id = '1-qH2pQIP9LNe3XO6mxLH9wlbbfTlT-8W'

# This creates the direct download URL
url = f'https://drive.google.com/uc?id={file_id}'

df = pd.read_csv(url)

# Display the first 5 rows to confirm it loaded correctly

print("Shape:",df.shape)
df.head()



## Introduction: Why Data Preparation?

> *Garbage in, garbage out.* Models cannot fix bad data.  
Data preparation typically consumes a large proportion of project time and directly impacts model accuracy and reliability.


## Step 1: Data Profiling

In [ ]:

# Inspect structure, types, and missingness
df.info()
display(df.isna().sum())
df.describe()

# Random sample 5 of rows
df.sample(5, random_state=0)


## Step 2: Data Cleaning



### Missing Values

In [ ]:
# Example strategies:
# 1) Deletion (listwise) — only when safe and small proportion missing
df_del = df.dropna(subset=['Age', 'Income'], how='any')
print("Listwise deletion shape:", df_del.shape)



In [ ]:
# 2) Deletion (column-wise) - when a large proportion is missing
# Calculate the percentage of missing values for each column
missing_percentage = df.isnull().sum() / len(df) * 100

# Identify columns with missing percentage greater than 10%
cols_to_drop = missing_percentage[missing_percentage > 10].index

# Drop these columns from the DataFrame
df_col_dropped = df.drop(columns=cols_to_drop)

print("Original shape:", df.shape)
print("Shape after dropping columns with >10% missing values:", df_col_dropped.shape)

In [ ]:
# 3) Deletion (listwise) - only when safe and small proportion missing
df_del_any = df.dropna(how='any')
print("Listwise deletion (any NaN) shape:", df_del_any.shape)

In [ ]:
# 4) Simple imputation
df_imp = df.copy()
df_imp['Age'] = df_imp['Age'].fillna(df_imp['Age'].median())
df_imp['Income'] = df_imp['Income'].fillna(df_imp['Income'].median())
df_imp[['Age','Income']].head(10)

In [ ]:
# 5) Predictive imputation can be done with models (skipped for brevity here)


### Inaccurate & Inconsistent Data

In [ ]:
df_imp['State'].unique()

In [ ]:
# Standardize State column (CA, California, Calif. -> CA)
state_map = {'California': 'CA', 'Calif.': 'CA'}
df_imp['State'] = df_imp['State'].replace(state_map)
df_imp['State'].unique()

In [ ]:
display(df_imp[['JoinDate','LastPurchaseDate']].head(10))

In [ ]:
# Ensure dates are parsed and consistent

df_imp['JoinDate'] = pd.to_datetime(df_imp['JoinDate'], errors='coerce')
df_imp['LastPurchaseDate'] = pd.to_datetime(df_imp['LastPurchaseDate'], errors='coerce')
df_imp.info()
df_imp[['JoinDate','LastPurchaseDate']].head(10)

In [ ]:
df_imp['Name'].head(10)

In [ ]:
# Trim/canonicalize text fields (example: Name)
df_imp['Name'] = df_imp['Name'].str.strip()

df_imp[['State','JoinDate','LastPurchaseDate','Name']].head(10)

### Duplicates

In [ ]:

# Count duplicates (all columns)
dup_count = df_imp.duplicated().sum()
print("Exact duplicate rows:", dup_count)

In [ ]:
# Identify potential dupes by a key subset (e.g., Name + JoinDate + State)
subset_dupes = df_imp.duplicated(subset=['Name','JoinDate','State']).sum()
print("Subset-based duplicate rows:", subset_dupes)

In [ ]:
# Drop exact duplicates
df_nodup = df_imp.drop_duplicates()
df_nodup.shape

## Step 3: Data Transformation

### Normalization (Min-Max) vs Standardization (Z-score)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scale_df = df_capped[['Income','TotalSpend_capped']].dropna().copy()

# Min-Max Scaling
mm = MinMaxScaler()
scale_df['Income_MinMax'] = mm.fit_transform(scale_df[['Income']])
scale_df.head(10)


In [ ]:
# Standardization
ss = StandardScaler()
scale_df['Income_Standard'] = ss.fit_transform(scale_df[['Income']])
scale_df.head(10)

### Data Aggregation (monthly sales example)

In [ ]:

# Aggregate by month using LastPurchaseDate and TotalSpend
agg = df_capped.dropna(subset=['LastPurchaseDate']).copy()
agg['Month'] = agg['LastPurchaseDate'].dt.to_period('M')
monthly_sales = agg.groupby('Month', as_index=False)['TotalSpend'].sum()

print("Monthly sales (sum of TotalSpend by last purchase month):")
display(monthly_sales)



In [ ]:
# Simple plot
plt.figure()
plt.plot(monthly_sales['Month'].astype(str), monthly_sales['TotalSpend'])
plt.xticks(rotation=45)
plt.title('Monthly Sales')
plt.xlabel('Month')
plt.ylabel('TotalSpend')
plt.tight_layout()
plt.show()


### Feature Engineering

In [ ]:
fe = df_capped.copy()

# Day of week from LastPurchaseDate
fe['DayOfWeek'] = fe['LastPurchaseDate'].dt.day_name()

# Days since last purchase (relative to "today")
today = pd.Timestamp.today().normalize()
fe['DaysSinceLastPurchase'] = (today - fe['LastPurchaseDate']).dt.days

# Simple CLV proxy: average monthly spend over recency (protect against div by zero)
fe['MonthsSinceLastPurchase'] = np.maximum(fe['DaysSinceLastPurchase'] / 30.0, 1.0)
fe['CLV_proxy'] = fe['TotalSpend'] / fe['MonthsSinceLastPurchase']

fe[['CustomerID','Name','DayOfWeek','DaysSinceLastPurchase','CLV_proxy']].head(10)


## Mini Case Study: Preparing Customer Data for Churn

In [ ]:
# Start from raw
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

raw = pd.read_csv(url)

# Profile the data
raw.head()
raw.info()
raw.describe()
raw.isnull().sum()

# 1) Make categorical names consistent and convert dates (stings) to datetime
raw['State'] = raw['State'].replace({'California':'CA','Calif.':'CA'})
raw['JoinDate'] = pd.to_datetime(raw['JoinDate'], errors='coerce')
raw['LastPurchaseDate'] = pd.to_datetime(raw['LastPurchaseDate'], errors='coerce')

# 2) Impute numeric columns (median)
for col in ['Age','Income']:
    raw[col] = raw[col].fillna(raw[col].median())

# 3) Handle missing LastPurchaseDate: fill with JoinDate as fallback (domain choice for demo)
raw['LastPurchaseDate'] = raw['LastPurchaseDate'].fillna(raw['JoinDate'])

# 4) Remove duplicates
clean = raw.drop_duplicates()

# 5) Feature engineering
today = pd.Timestamp.today().normalize()
clean['DaysSinceLastPurchase'] = (today - clean['LastPurchaseDate']).dt.days
clean['TenureDays'] = (today - clean['JoinDate']).dt.days
clean['AvgMonthlySpend'] = clean['TotalSpend'] / np.maximum(clean['TenureDays']/30.0, 1.0)

# Simple CLV proxy: AvgMonthlySpend * 6 months horizon
clean['CustomerLifetimeValue_6mo'] = clean['AvgMonthlySpend'] * 6

# 6) Scaling selected features (for model input)
for_scale = clean[['Income','TotalSpend','DaysSinceLastPurchase','TenureDays','CustomerLifetimeValue_6mo']].copy()
scaler = StandardScaler()
scaled = pd.DataFrame(scaler.fit_transform(for_scale), columns=[c + "_z" for c in for_scale.columns])

model_input = pd.concat([clean[['CustomerID','Churn','State']], scaled], axis=1)
model_input.head(10)



## Conclusion & Key Takeaways

- Data preparation is iterative: profile → clean → transform → engineer → (repeat as needed).  
- Document your assumptions and choices (e.g., why you imputed a value a certain way).  
- High-quality preparation underpins trustworthy, impactful models.
